In [21]:
import pandas as pd
from pathlib import Path
from Bio import AlignIO
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Blast import NCBIWWW, NCBIXML
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import re

from src.sup_data_to_fasta import load_xlsx, transform_raw_dfs_to_queries

sup5_file_path = Path("data/Liu_sup5_data.xlsx")

# Análise Exploratória dos Dados Suplementares 5

## Sobre

Este notebook contém uma análise exploratória dos dados fornecidos por Liu, et al. No [suplemento 5](https://www.nature.com/articles/s41467-023-43632-1#additional-information
) com o objetivo de averiguar os tratamentos executados no artigo

### Objetivo

* Verificar a origem das tags dos pares sRNA/mRNA dos dados suplementares.
* Ganhar mais insight sobre as interações medidas.



In [39]:
import pandas as pd
from pathlib import Path

def load_alignments():
    output_dir = Path("output")
    alignments_files = output_dir.glob("*alignments_results.tsv")
    
    exp_word_size_list = []

    alignments_df_list = []
    for file_path in alignments_files:
        prefix = file_path.stem.split("_")[0]
        df = pd.read_csv(file_path, sep='\t')
        word_sz = prefix.split("w")[1]
        experiment = prefix.split("-")[0]
        exp_word_size_list.append((experiment, word_sz))
        df["experiment"] = experiment
        df["word_sz"] = word_sz
        alignments_df_list.append(df)
    
    alignments_dict = {(exp, word_sz): df for (exp, word_sz), df in zip(exp_word_size_list, alignments_df_list)}
    return alignments_dict, exp_word_size_list


In [40]:
alignments_dict, exp_word_size_list = load_alignments()

In [25]:
def load_queries():
    sup5_file_path = Path("data/Liu_sup5_data.xlsx")
    dict = load_xlsx(sup5_file_path)
    queries_dict = transform_raw_dfs_to_queries(dict)
    queries_dict = {k.split('-')[0]:v for k,v in queries_dict.items()}
    return queries_dict

In [26]:
queries_dict = load_queries()

In [31]:
queries_df = queries_dict['EP']
alignments_df = alignments_dict[('EP', '11')]
print(f"#queries {len(queries_df)}")
print(f"#alignments {len(alignments_df)}")

#queries 420
#alignments 6485


In [41]:
def join_dfs(queries_df, alignments_df):
    joined_df = pd.merge(queries_df, alignments_df, left_on='name', right_on='qseqid', how='left')
    return joined_df

In [42]:
aligned_queries_dict = {}
for (exp, word_sz) in exp_word_size_list:
    queries_df = queries_dict[exp]
    alignments_df = alignments_dict[(exp, word_sz)]
    joined_df = join_dfs(queries_df, alignments_df)
    aligned_queries_dict[(exp, word_sz)] = joined_df

dict_keys([('SP', '11'), ('ESP', '10'), ('ESP', '9'), ('SP', '7'), ('ESP', '8'), ('ESP', '11'), ('SP', '10'), ('EP', '9'), ('EP', '8'), ('SP', '9'), ('EP', '11'), ('SP', '8'), ('ESP', '7'), ('EP', '10'), ('EP', '7')])